In [120]:
# importando pacotes 
import matplotlib.pyplot as plt
import import_ipynb 
import pandas as pd
import numpy as np
import warnings
from FuncaoProposta1 import retorna_lista
from sklearn.metrics import mean_squared_error
from math import sqrt
warnings.filterwarnings('ignore')


In [121]:
nome_pasta_medias = ['0 ORIGINAL','1 PRESS_SAT','2 MEDIA_05','3 MEDIA_07','4 MEDIA_10',
                     '5 MEDIA_15','6 MEDIA_20','7 MEDIA_25','8 MEDIA_PONTO' ]

i = 8
ano  = 2022
nome = nome_pasta_medias[i].split(' ')[-1]
nome

'MEDIA_PONTO'

In [122]:
# DEFININDO COLUNAS 
colunas  = ['date', 'hour', 'nseg', 'lon', 'lat', 'u', 'v', 'press', 'qi', 'BFP_g', 'BFP_l', 'u_nwp_AMV', 'u_nwp_BFP_g', 'u_nwp_BFP_l', 'v_nwp_AMV', 'v_nwp_BFP_g', 'v_nwp_BFP_l','spd_diff_AMV', 'spd_diff_BFP_g', 'spd_diff_BFP_l', 'vctr_diff_AMV', 'vctr_diff_BFP_g', 'vctr_diff_BFP_l', 'BFP_qc', 'no name']

# DEFININDO COLUNAS QUE SERÃO LIDAS 
colunas_lidas = ['date','hour', 'lon', 'lat', 'u' ,'v' , 'press','qi','u_nwp_BFP_l', 'v_nwp_BFP_l'] #'u_nwp_AMV','v_nwp_AMV' ] 

# DEFININDO CAMINHO ONDE ESTÃO OS ARQUIVOS 
#caminho = '/home/leonardo/Downloads/DADOS_INPE_SAIDAS_MODELO 2022/'
caminho = f'/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/ARQUIVOS BFP/{ano}/{nome_pasta_medias[i]}/'

caminho_salvar =f'/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/ARQUIVOS BFP/{ano}/IMAGENS/'
caminho_salvar_txt =f'/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/ARQUIVOS BFP/{ano}/TXT/'
caminho_salvar_txt_alt= f'/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/ARQUIVOS BFP/{ano}/ALTURA/' 
arquivo = retorna_lista(caminho)


In [123]:
# --------------- DEFININDO FUNCOES ---------------

def root_mean_square(var, ref):
    return sqrt(mean_squared_error(var, ref))
def mean_bias_error(true, pred):
    mbe_loss = np.sum(true - pred)/true.size
    return mbe_loss
def rms_plot(s1, s1l, s2, s2l, s3, s3l, s4, s4l, nome, caminho_salvar,
    ymin=None, ymax=None, figout=None, xticks=None,
    title='RMSE 2018-2022 - componentes u/v do vento - AMV-DSA (IR) x Reanalise ERA5'):

    # import matplotlib.dates as mdates
    # import datetime as dt
    # now = dt.datetime.now()
    # then = now + dt.timedelta(days=100)
    # days = mdates.drange(now,then,dt.timedelta(days=1))

    fig, ax = plt.subplots(1,1)

    plt.figure(figsize=(12,3))
    plt.plot(xticks, s1, label=s1l, color='red')
    if s2: plt.plot(xticks, s2, label=s2l, color='green')
    if s3: plt.plot(xticks, s3, label=s3l, color='red', ls='--')
    if s4: plt.plot(xticks, s4, label=s4l, color='green', ls='--')
    plt.title(title)
    plt.ylabel('m/s')
    plt.ylim(ymin, ymax)
    plt.xlabel('months')
    plt.xticks(rotation=90)
    
    if not xticks:
        plt.xticks(range(12*4))
    else:
        print(xticks)
        # Set number of ticks for x-axis
        ax.set_xticks(range(len(xticks)))
        # Set ticks labels for x-axis
        ax.set_xticklabels(xticks, rotation='vertical', fontsize=18)

    plt.legend(loc='best')

    if figout:
        plt.savefig(f'{caminho_salvar}/{figout}_{nome}.png', dpi=100, bbox_inches='tight')
    else:
        plt.show()
def rms_plot_alt(s1, s1l, s2, s2l, s3, s3l, nome, caminho_salvar,
    ymin=None, ymax=None, figout=None, xticks=None,
    title='RMSE 2018-2022 - componentes u/v do vento - AMV-DSA (IR) x Reanalise ERA5'):
    
    fig, ax = plt.subplots(1,1)

    plt.figure(figsize=(12,3))
    plt.plot(xticks, s1, label=s1l, color='pink') # nuvens baixas
    if s2: plt.plot(xticks, s2, label=s2l, color='green') #NUVENS MEDIAS
    if s3: plt.plot(xticks, s3, label=s3l, color='blue') #, ls='--') #NUVENS ALTAS
    #if s4: plt.plot(xticks, s4, label=s4l, color='', ls='--')
    plt.title(title)
    plt.ylabel('m/s')
    plt.ylim(ymin, ymax)
    plt.xlabel('meses')
    plt.xticks(rotation=90)
    
    if not xticks:
        plt.xticks(range(12*4))
    else:
        print(xticks)
        # Set number of ticks for x-axis
        ax.set_xticks(range(len(xticks)))
        # Set ticks labels for x-axis
        ax.set_xticklabels(xticks, rotation='vertical', fontsize=18)

    plt.legend(loc='best')

    if figout:
        plt.savefig(f'{caminho_salvar}/{figout}_{nome}_altura.png', dpi=100, bbox_inches='tight')
    else:
        plt.show()


In [124]:
# CRIANDO FUNÇÃO  PARA LER ARQUIVOS E REALIZAR AS MEDIAS DE RMSE E BIAS

# VARIAVEIS GLOBAIS
stats_rms_eq = {'u':[], 'v':[], 'nobs':[]}
stats_rms_sh = {'u':[], 'v':[], 'nobs':[]}
stats_bias_eq = {'u':[], 'v':[], 'nobs':[]}
stats_bias_sh = {'u':[], 'v':[], 'nobs':[]}
# criando outro dict para altura de nuvens 
nuvens_baixas_rmse = {'u' :[], 'v':[],'nobs':[] } #850 A 100
nuvens_baixas_bias = {'u' :[], 'v':[],'nobs':[] }
nuvens_baixas1_rmse = {'u' :[], 'v':[],'nobs':[] } #700 A 850
nuvens_baixas1_bias = {'u' :[], 'v':[],'nobs':[] }

nuvens_medias_rmse = {'u' :[], 'v':[],'nobs':[] } #550 A 699
nuvens_medias_bias = {'u' :[], 'v':[],'nobs':[] }
nuvens_medias1_rmse = {'u' :[], 'v':[],'nobs':[] } # 400 A 549
nuvens_medias1_bias = {'u' :[], 'v':[],'nobs':[] }

nuvens_altas_rmse = {'u' :[], 'v':[],'nobs':[] } #250 A 399
nuvens_altas_bias = {'u' :[], 'v':[],'nobs':[] }
nuvens_altas1_rmse = {'u' :[], 'v':[],'nobs':[] } # 100 A 249
nuvens_altas1_bias = {'u' :[], 'v':[],'nobs':[] }


xlabels = []

ARQUIVO_APPENDI = []

tam_pasta = len(arquivo)
# entrando nas pastas 01,03,05 etc 
for cont1 in range(0, tam_pasta, 1):
    #retornando caminho dentro das pastas 
    pasta_mes = retorna_lista(arquivo[cont1])
    tam_pasta_mes = len(pasta_mes)
    
    #xlabels.append(str(ano)+'-'+str(mes).zfill(2)+'-'+str(dia).zfill(2))
    
    data = pd.DataFrame()
    # fazendo caminho dentro das pastas pegando os arquivos dentro das pastas
    for cont2 in range(0, tam_pasta_mes,1):
        #LENDO ARQUIVOS DENTRO DAS PASTAS
        #first = True
        arquivo_txt = pd.read_csv(pasta_mes[cont2], sep = ' ',
                                  skiprows= 0, header = 0)        
        arquivo_txt.columns = colunas

        arquivo_modi = []
        arquivo_modi =  arquivo_txt[colunas_lidas].copy()
        
        # SEPARANDO POR QI 
        arquivo_modi_2 = arquivo_modi[arquivo_modi['qi'] > 0.7]
        
        #if first:
        #    data = arquivo_modi_2
        #    first = False
        #else:
        #    data = np.vstack((data,arquivo_modi_2))
            
        if data.empty:
            data = arquivo_modi_2.copy()
        else:
            data = pd.concat([data, arquivo_modi_2], ignore_index=True)
            
    # SEPARANDO POR LATITUDES EQUATORIAL E HS
    #data.columns = colunas_lidas

    EQ  = data[data['lat'] > -20]
    SH  = data[data['lat'] <= -20]
    # SEPARANDO POR ALTURA

    #NB  = data[data['press'] >= 750]
    NB  = data[(data['press'] >= 850) & (data['press'] <= 1000)]
    NB1  = data[(data['press'] >700) & (data['press']  < 850)]

    NM  = data[(data['press'] >= 550) & (data['press']  < 700)]
    NM1  = data[(data['press'] >= 400) & (data['press'] < 550)]
    
    NA  = data[(data['press'] >= 250) & (data['press']   < 400)]
    NA1  = data[(data['press']>= 100) & (data['press'] < 250)]
    #NA  = data[data['press'] < 400]
    
    # --------------------------
    #s = SH
    #u, v = data[s, 'u'], data[s, 'v']
    #u_n, v_n = data[s, 'u_nwp_AMV'],data[s, 'v_nwp_AMV'] 
    
    s = SH
    u, v = s['u'], s['v']
    #u_n, v_n = s['u_nwp_AMV'], s['v_nwp_AMV']
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u, rmse_v = root_mean_square(u, u_n), root_mean_square(v, v_n)
    bias_u, bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    nobs = len(s)
    
    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    stats_rms_sh['u'].append(rmse_u)
    stats_rms_sh['v'].append(rmse_v)
    stats_rms_sh['nobs'].append(nobs)
    stats_bias_sh['u'].append(bias_u)
    stats_bias_sh['v'].append(bias_v)
    
    s = EQ
   #u, v = data[s, 'u'], data[s, 'v']
   #u_n, v_n = data[s, 'u_nwp_AMV'],data[s, 'v_nwp_AMV'] 
    u, v = s['u'], s['v']
    #u_n, v_n = s[ 'u_nwp_AMV'],s['v_nwp_AMV'] 
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u,rmse_v =  root_mean_square(u, u_n ),root_mean_square(v, v_n )
    bias_u,bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    #nobs = data[s,0].size
    nobs = len(s)

    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    stats_rms_eq['u'].append(rmse_u)
    stats_rms_eq['v'].append(rmse_v)
    stats_rms_eq['nobs'].append(nobs)
    stats_bias_eq['u'].append(bias_u)
    stats_bias_eq['v'].append(bias_v)

    extract_date = lambda x: (x[:4], x[4:6])
    ano,mes = extract_date(str(np.mean(data['date'][0:100]))) 
    xlabels.append(str(ano)+'-'+str(mes).zfill(2))


     # POR ALTURA 
    # NUVENS BAIXAS DE 850 HPA A 1000 HPA
    s = NB
    u, v = s['u'], s['v']
    #u_n, v_n = s['u_nwp_AMV'], s['v_nwp_AMV']
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u, rmse_v = root_mean_square(u, u_n), root_mean_square(v, v_n)
    bias_u, bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    nobs = len(s)
    
    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    nuvens_baixas_rmse['u'].append(rmse_u)
    nuvens_baixas_rmse['v'].append(rmse_v)
    nuvens_baixas_rmse['nobs'].append(nobs)
    nuvens_baixas_bias['u'].append(bias_u)
    nuvens_baixas_bias['v'].append(bias_v)
    
    # NUVENS BAIXAS DE 700 HPA A 849 
    s = NB1
    u, v = s['u'], s['v']
    #u_n, v_n = s['u_nwp_AMV'], s['v_nwp_AMV']
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u, rmse_v = root_mean_square(u, u_n), root_mean_square(v, v_n)
    bias_u, bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    nobs = len(s)
    
    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    nuvens_baixas1_rmse['u'].append(rmse_u)
    nuvens_baixas1_rmse['v'].append(rmse_v)
    nuvens_baixas1_rmse['nobs'].append(nobs)
    nuvens_baixas1_bias['u'].append(bias_u)
    nuvens_baixas1_bias['v'].append(bias_v)


    # NUVENS MEDIAS  550 A 699
    s = NM
    u, v = s['u'], s['v']
    #u_n, v_n = s['u_nwp_AMV'], s['v_nwp_AMV']
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u, rmse_v = root_mean_square(u, u_n), root_mean_square(v, v_n)
    bias_u, bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    nobs = len(s)
    
    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    nuvens_medias_rmse['u'].append(rmse_u)
    nuvens_medias_rmse['v'].append(rmse_v)
    nuvens_medias_rmse['nobs'].append(nobs)
    nuvens_medias_bias['u'].append(bias_u)
    nuvens_medias_bias['v'].append(bias_v)
  
    # NUVENS MEDIAS DE 400 A 549
    s = NM1
    u, v = s['u'], s['v']
    #u_n, v_n = s['u_nwp_AMV'], s['v_nwp_AMV']
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u, rmse_v = root_mean_square(u, u_n), root_mean_square(v, v_n)
    bias_u, bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    nobs = len(s)
    
    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    nuvens_medias1_rmse['u'].append(rmse_u)
    nuvens_medias1_rmse['v'].append(rmse_v)
    nuvens_medias1_rmse['nobs'].append(nobs)
    nuvens_medias1_bias['u'].append(bias_u)
    nuvens_medias1_bias['v'].append(bias_v)
  
    # NUVENS ALTAS DE 250 A 399
    s = NA
    u, v = s['u'], s['v']
    #u_n, v_n = s['u_nwp_AMV'], s['v_nwp_AMV']
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u, rmse_v = root_mean_square(u, u_n), root_mean_square(v, v_n)
    bias_u, bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    nobs = len(s)
    
    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    nuvens_altas_rmse['u'].append(rmse_u)
    nuvens_altas_rmse['v'].append(rmse_v)
    nuvens_altas_rmse['nobs'].append(nobs)
    nuvens_altas_bias['u'].append(bias_u)
    nuvens_altas_bias['v'].append(bias_v)

    # NUVENS ALTAS DE 100 A 249
    s = NA1
    u, v = s['u'], s['v']
    #u_n, v_n = s['u_nwp_AMV'], s['v_nwp_AMV']
    u_n, v_n = s['u_nwp_BFP_l'], s['v_nwp_BFP_l']

    rmse_u, rmse_v = root_mean_square(u, u_n), root_mean_square(v, v_n)
    bias_u, bias_v = mean_bias_error(u_n, u), mean_bias_error(v_n, v)
    nobs = len(s)
    
    # CRIANDO DATA FRAME PARA ARMAZENAR OS DADOS DE MEDIAS
    nuvens_altas1_rmse['u'].append(rmse_u)
    nuvens_altas1_rmse['v'].append(rmse_v)
    nuvens_altas1_rmse['nobs'].append(nobs)
    nuvens_altas1_bias['u'].append(bias_u)
    nuvens_altas1_bias['v'].append(bias_v)


# CRIANDO DATA FRAME E ARMAZENANDO
BIAS_RMSE_EQ_SH = pd.DataFrame(data={'rms_eq_u':stats_rms_eq['u'], 'rms_sh_u':stats_rms_sh['u'], 
                                   'bias_eq_u':stats_bias_eq['u'], 'bias_sh_u':stats_bias_sh['u'],
                                   'rms_eq_v':stats_rms_eq['v'], 'rms_sh_v':stats_rms_sh['v'], 
                                   'bias_eq_v':stats_bias_eq['v'], 'bias_sh_v':stats_bias_sh['v'],
                                   'nobs_eq' :stats_rms_eq['nobs'],'nobs_sh':stats_rms_sh['nobs']
                                   },
                                    index =xlabels  )
                             
BIAS_RMSE_EQ_SH.to_csv(f'{caminho_salvar_txt}/BIAS_RMSE_EQ_SH_{ano}_{nome}')

# CRIANDO DATA FRAME POR ALTURA E ARMAZENANDO
BIAS_RMSE_SEP_ALTURA = pd.DataFrame(data={'rmse_NB_850_1000_u':nuvens_baixas_rmse['u'], 'rmse_NB_850_1000_v': nuvens_baixas_rmse['v'],
                                          'bias_NB_850_1000_u':nuvens_baixas_bias['u'],'bias_NB_850_1000_v':nuvens_baixas_bias['v'],
                                          'nobs_NB_850_1000':nuvens_baixas_rmse['nobs'],

                                          'rmse_NB_700_849_u':nuvens_baixas1_rmse['u'], 'rmse_NB_700_849_v': nuvens_baixas1_rmse['v'],
                                          'bias_NB_700_849_u':nuvens_baixas1_bias['u'],'bias_NB_700_849_v':nuvens_baixas1_bias['v'],
                                          'nobs_NB_700_849':nuvens_baixas1_rmse['nobs'],

                                          'rmse_NM_550_699_u':nuvens_medias_rmse['u'], 'rmse_NM_550_699_v': nuvens_medias_rmse['v'],
                                          'bias_NM_550_699_u':nuvens_medias_bias['u'],'bias_NM_550_699_v':nuvens_medias_bias['v'],
                                          'nobs_NM_550_699':nuvens_medias_rmse['nobs'],

                                          'rmse_NM_400_549_u':nuvens_medias1_rmse['u'], 'rmse_NM_400_549_v': nuvens_medias1_rmse['v'],
                                          'bias_NM_400_549_u':nuvens_medias1_bias['u'],'bias_NM_400_549_v':nuvens_medias1_bias['v'],
                                          'nobs_NM_400_549':nuvens_medias1_rmse['nobs'],

                                          'rmse_NA_250_399_u':nuvens_altas_rmse['u'], 'rmse_NA_250_399_v': nuvens_altas_rmse['v'],
                                          'bias_NA_250_399_u':nuvens_altas_bias['u'],'bias_NA_250_399_v':nuvens_altas_bias['v'],
                                          'nobs_NA_250_399':nuvens_altas_rmse['nobs'],

                                          'rmse_NA_100_249_u':nuvens_altas1_rmse['u'], 'rmse_NA_100_249_v': nuvens_altas1_rmse['v'],
                                          'bias_NA_100_249_u':nuvens_altas1_bias['u'],'bias_NA_100_249_v':nuvens_altas1_bias['v'],
                                          'nobs_NA_100_249':nuvens_altas1_rmse['nobs']

                                        },index =xlabels  )

BIAS_RMSE_SEP_ALTURA.to_csv(f'{caminho_salvar_txt_alt}/BIAS_RMSE_alt_{ano}_{nome}')


#lines = BIAS_RMSE.plot.line()

# PLOTS

In [125]:
## ================== PLOTS ==================
#rms_plot(s1=stats_rms_eq['u'][:], s1l='u-comp EQ', s2=stats_rms_eq['v'][:], s2l='v-comp EQ',
#         s3=stats_rms_sh['u'][:], s3l='u-comp SH', s4=stats_rms_sh['v'][:], s4l='v-comp SH',
#         title='RMS', figout='RMSVD', nome = nome, caminho_salvar=caminho_salvar,
#         xticks=xlabels)
#rms_plot(s1=stats_bias_eq['u'][:], s1l='u-comp EQ', s2=stats_bias_eq['v'][:], s2l='v-comp EQ',
#         s3=stats_bias_sh['u'][:], s3l='u-comp SH', s4=stats_bias_sh['v'][:], s4l='v-comp SH',
#         title='SPEED BIAS', ymin=-4, ymax=4, figout='BIAS',nome = nome, caminho_salvar=caminho_salvar,
#         xticks=xlabels)
#rms_plot(s1=stats_rms_eq['nobs'][:], s1l='nobs EQ', s2=None, s2l=None,
#         s3=stats_rms_sh['nobs'][:], s3l='nobs SH', s4=None, s4l=None,
#         title='Num. Observations', figout='NOBS',nome = nome, caminho_salvar=caminho_salvar,
#         xticks=xlabels)
#
## ---------------------- PLOTANDO GRAFICOS DE ALTURA 
## COMPONENTE U DO VENTO -- RMSE
#rms_plot_alt(s1 =nuvens_baixas_rmse['u'][:],s1l='Nuvens Baixas', s2=nuvens_medias_rmse['u'][:], s2l='Nuvens Medias',
#         s3=nuvens_altas_rmse['u'][:], s3l='Nuvens Altas',nome = nome, figout='RMSE', title= "Componente U do vento RMSE", caminho_salvar= caminho_salvar,xticks = xlabels ) 
## COMPONENTE V DO VENTO -- RMSE
#rms_plot_alt(s1 =nuvens_baixas_rmse['v'][:],s1l='Nuvens Baixas', s2=nuvens_medias_rmse['v'][:], s2l='Nuvens Medias',
#         s3=nuvens_altas_rmse['v'][:], s3l='Nuvens Altas',nome = nome, figout='RMSE', title= "Componente V do vento RMSE", caminho_salvar= caminho_salvar,xticks = xlabels ) 
## COMPONENTE U DO VENTO -- BIAS
#rms_plot_alt(s1 =nuvens_baixas_bias['u'][:],s1l='Nuvens Baixas', s2=nuvens_medias_bias['u'][:], s2l='Nuvens Medias',
#         s3=nuvens_altas_bias['u'][:], s3l='Nuvens Altas',nome = nome, figout=' BIAS', title= "Componente U do vento BIAS", caminho_salvar= caminho_salvar,xticks = xlabels )  
## COMPONENVE V DO VENTO -- BIAS
#rms_plot_alt(s1 =nuvens_baixas_bias['v'][:],s1l='Nuvens Baixas', s2=nuvens_medias_bias['v'][:], s2l='Nuvens Medias',
#         s3=nuvens_altas_bias['v'][:], s3l='Nuvens Altas',nome = nome, figout=' BIAS', title= "Componente V do vento BIAS", caminho_salvar= caminho_salvar,xticks = xlabels ) 
## NUMERO DE OBSERVAÇÕES 
#rms_plot_alt(s1 =nuvens_baixas_rmse['nobs'][:],s1l='Nuvens Baixas', s2=nuvens_medias_rmse['nobs'][:],
#              s2l='Nuvens Medias',s3=nuvens_altas_rmse['nobs'][:], s3l='Nuvens Altas',nome = nome, figout='NOBS', title= "Número de observações", caminho_salvar= caminho_salvar,xticks = xlabels ) 

In [126]:

#lines = BIAS_RMSE.plot.line()
#lins = BIAS_RMSE_SEP_ALTURA.plot.line()